In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [18]:
pip install poppler

ERROR: Could not find a version that satisfies the requirement poppler (from versions: none)
ERROR: No matching distribution found for poppler
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install pandas requests beautifulsoup4 scrapy datasets pdfminer.six clean-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 2.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 70.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.0 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=309618ea2b8e60430a628a361af55640cdf7b89f067903ff7b66cba94d

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import scrapy
from scrapy.crawler import CrawlerProcess
from datasets import load_dataset
from pdfminer.high_level import extract_text
import os
import re
from cleantext import clean

# Languages and target sample size per language
LANGUAGES = ["Hindi", "Marathi", "Sindhi", "Gujarati", "Bengali", "Tamil", "Kannada"]
TARGET_PER_LANG = 750000  # Aiming for ~5.25 million total
MIN_WORDS = 50

# Function to filter text by word count
def filter_text_length(text):
    if not isinstance(text, str):  # Handle non-string inputs
        return False
    words = text.split()
    return len(words) >= MIN_WORDS

# Function 1: Load from Public Datasets
def load_from_datasets(output_file="dataset_corpus.csv"):
    print("Loading from public datasets...")
    corpus = []
    
    # Try IndicCorp (corrected name or fallback)
    try:
        # Note: IndicCorp might need manual download from AI4Bharat; using OSCAR as a fallback
        dataset = load_dataset("oscar", "unshuffled_deduplicated_hi", split="train")  # Hindi example
        df = pd.DataFrame(dataset)
        filtered_df = df[df['text'].apply(filter_text_length)].sample(n=min(TARGET_PER_LANG, len(df)), random_state=42)
        filtered_df = filtered_df[['text']].assign(language="Hindi")
        corpus.append(filtered_df)
        print("Loaded Hindi from OSCAR")
    except Exception as e:
        print(f"Error loading OSCAR for Hindi: {e}")
    
    # Add other languages from OSCAR (example for Tamil)
    try:
        dataset = load_dataset("oscar", "unshuffled_deduplicated_ta", split="train")
        df = pd.DataFrame(dataset)
        filtered_df = df[df['text'].apply(filter_text_length)].sample(n=min(TARGET_PER_LANG, len(df)), random_state=42)
        filtered_df = filtered_df[['text']].assign(language="Tamil")
        corpus.append(filtered_df)
        print("Loaded Tamil from OSCAR")
    except Exception as e:
        print(f"Error loading OSCAR for Tamil: {e}")

    # Manually add mappings for others if OSCAR works (Marathi: mr, Gujarati: gu, Bengali: bn, Kannada: kn)
    oscar_langs = {"mr": "Marathi", "gu": "Gujarati", "bn": "Bengali", "kn": "Kannada"}
    for code, lang in oscar_langs.items():
        try:
            dataset = load_dataset("oscar", f"unshuffled_deduplicated_{code}", split="train")
            df = pd.DataFrame(dataset)
            filtered_df = df[df['text'].apply(filter_text_length)].sample(n=min(TARGET_PER_LANG, len(df)), random_state=42)
            filtered_df = filtered_df[['text']].assign(language=lang)
            corpus.append(filtered_df)
            print(f"Loaded {lang} from OSCAR")
        except Exception as e:
            print(f"Error loading OSCAR for {lang}: {e}")

    # Sindhi might not be in OSCAR easily; skip or source elsewhere
    
    # Combine and save
    if corpus:
        combined_df = pd.concat(corpus).drop_duplicates(subset="text")
        combined_df.to_csv(output_file, index=False)
        print(f"Saved {len(combined_df)} samples from datasets to {output_file}")
        return combined_df
    else:
        print("No datasets loaded successfully.")
        return pd.DataFrame(columns=["text", "language"])  # Return empty DataFrame

# Function 2: Web Scraping
def scrape_from_web(output_file="scraped_corpus.csv"):
    print("Scraping from web...")
    sites = {
        "Hindi": ["https://hindi.bbc.com", "https://www.bhaskar.com"],
        "Marathi": ["https://lokmat.com", "https://maharashtratimes.com"],
        "Sindhi": ["https://awamiawaz.pk"],
        "Gujarati": ["https://divyabhaskar.co.in", "https://sandesh.com"],
        "Bengali": ["https://anandabazar.com", "https://eisamay.com"],
        "Tamil": ["https://dinamalar.com", "https://dailythanthi.com"],
        "Kannada": ["https://prajavani.net", "https://vijaykarnataka.com"]
    }
    
    corpus = []
    headers = {"User-Agent": "Mozilla/5.0"}
    
    for lang, urls in sites.items():
        lang_texts = []
        for url in urls:
            try:
                response = requests.get(url, headers=headers, timeout=10)
                soup = BeautifulSoup(response.content, "html.parser")
                paragraphs = soup.find_all("p")
                for p in paragraphs:
                    text = clean(p.get_text(), no_line_breaks=True, no_urls=True, no_emails=True)
                    if filter_text_length(text):
                        lang_texts.append({"text": text, "language": lang})
            except Exception as e:
                print(f"Error scraping {url}: {e}")
        
        lang_df = pd.DataFrame(lang_texts).sample(n=min(TARGET_PER_LANG, len(lang_texts)), random_state=42)
        corpus.append(lang_df)
    
    if corpus:
        combined_df = pd.concat(corpus).drop_duplicates(subset="text")
        combined_df.to_csv(output_file, index=False)
        print(f"Saved {len(combined_df)} samples from web to {output_file}")
        return combined_df
    return pd.DataFrame(columns=["text", "language"])

# Function 3: OCR from PDFs
def extract_from_pdfs(pdf_dir="pdfs", output_file="pdf_corpus.csv"):
    print("Extracting from PDFs...")
    corpus = []
    
    for lang in LANGUAGES:
        lang_texts = []
        lang_dir = os.path.join(pdf_dir, lang.lower())
        if not os.path.exists(lang_dir):
            print(f"No PDFs found for {lang}, skipping...")
            continue
        
        for pdf_file in os.listdir(lang_dir):
            if pdf_file.endswith(".pdf"):
                try:
                    text = extract_text(os.path.join(lang_dir, pdf_file))
                    paragraphs = text.split("\n\n")
                    for para in paragraphs:
                        cleaned_text = clean(para, no_line_breaks=True, no_urls=True, no_emails=True)
                        if filter_text_length(cleaned_text):
                            lang_texts.append({"text": cleaned_text, "language": lang})
                except Exception as e:
                    print(f"Error processing {pdf_file}: {e}")
        
        lang_df = pd.DataFrame(lang_texts).sample(n=min(TARGET_PER_LANG, len(lang_texts)), random_state=42)
        corpus.append(lang_df)
    
    if corpus:
        combined_df = pd.concat(corpus).drop_duplicates(subset="text")
        combined_df.to_csv(output_file, index=False)
        print(f"Saved {len(combined_df)} samples from PDFs to {output_file}")
        return combined_df
    return pd.DataFrame(columns=["text", "language"])

# Main function to build the corpus
def build_corpus(output_file="indic_corpus.csv"):
    print("Building the corpus...")
    
    dataset_df = load_from_datasets()
    scraped_df = scrape_from_web()
    pdf_df = extract_from_pdfs()
    
    all_dfs = [df for df in [dataset_df, scraped_df, pdf_df] if not df.empty]
    if not all_dfs:
        print("No data collected!")
        return
    
    combined_df = pd.concat(all_dfs).drop_duplicates(subset="text")
    
    balanced_corpus = []
    for lang in LANGUAGES:
        lang_df = combined_df[combined_df['language'] == lang]
        sampled_df = lang_df.sample(n=min(TARGET_PER_LANG, len(lang_df)), random_state=42)
        balanced_corpus.append(sampled_df)
    
    final_df = pd.concat(balanced_corpus).sample(frac=1, random_state=42)
    
    if len(final_df) < 5000000:
        print(f"Warning: Only {len(final_df)} samples collected, below 5 million!")
    else:
        print(f"Success: Collected {len(final_df)} samples!")
    
    final_df.to_csv(output_file, index=False)
    print(f"Corpus saved to {output_file}")

if __name__ == "__main__":
    build_corpus()

In [4]:
!pip install langdetect

  Using cached langdetect-1.0.9-py3-none-any.whl


In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datasets import load_dataset
from pdfminer.high_level import extract_text
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import cv2
import numpy as np
import os
from cleantext import clean
import gc
from langdetect import detect_langs

# Languages and target sample size per language
LANGUAGES = ["Hindi", "Marathi", "Sindhi", "Gujarati", "Bengali", "Tamil", "Kannada"]
TARGET_PER_LANG = 250000  # Reduced from 750000 to prevent memory issues
MIN_WORDS = 50

# Function to filter text by word count
def filter_text_length(text):
    if not isinstance(text, str):
        return False
    words = text.split()
    return len(words) >= MIN_WORDS

# Preprocess image for OCR
def preprocess_image(image):
    """Preprocess scanned images to improve OCR accuracy."""
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    return Image.fromarray(processed_image)

# Extract text from PDF using OCR with better memory management
def extract_text_from_pdf(pdf_path, batch_size=10):  # Reduced batch size
    """Extract text from scanned PDFs using OCR in batches."""
    try:
        # Convert only the first 20 pages to prevent memory issues
        images = convert_from_path(pdf_path, dpi=200, first_page=1, last_page=20)  
        text = ""
        for i in range(0, len(images), batch_size):
            batch_images = images[i:i + batch_size]
            for image in batch_images:
                processed_image = preprocess_image(image)
                text += pytesseract.image_to_string(
                    processed_image,
                    lang='hin+mar+ben+tam+kan+eng+urd+chi',
                    config="--psm 6 --oem 1"
                ) + "\n"
            # Force garbage collection after each batch
            del batch_images
            processed_image = None
            gc.collect()
        return text.strip()
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return ""

# Detect language based on Unicode ranges
def detect_language_unicode(text):
    """Detects and labels language based on Unicode ranges."""
    lang_counts = {
        'Hindi': sum(1 for char in text if '\u0900' <= char <= '\u097F'),  # Devanagari
        'Marathi': sum(1 for char in text if '\u0900' <= char <= '\u097F'), # Devanagari (overlap with Hindi)
        'Sindhi': sum(1 for char in text if '\u0600' <= char <= '\u06FF'),  # Arabic script
        'Gujarati': sum(1 for char in text if '\u0A80' <= char <= '\u0AFF'),
        'Bengali': sum(1 for char in text if '\u0980' <= char <= '\u09FF'),
        'Tamil': sum(1 for char in text if '\u0B80' <= char <= '\u0BFF'),
        'Kannada': sum(1 for char in text if '\u0C80' <= char <= '\u0CFF'),
        'Urdu': sum(1 for char in text if '\u0600' <= char <= '\u06FF'),    # Arabic script
        'Chinese': sum(1 for char in text if '\u4E00' <= char <= '\u9FFF'),
        'English': sum(1 for char in text if '\u0041' <= char <= '\u007A' or '\u0061' <= char <= '\u007A')
    }
    
    total_chars = sum(lang_counts.values())
    if total_chars == 0:
        return None, {}, {}
    
    lang_probs = {lang: count / total_chars for lang, count in lang_counts.items()}
    top_lang = max(lang_probs, key=lang_probs.get)
    
    if top_lang in LANGUAGES:
        return top_lang, lang_probs, {top_lang: text}
    return None, lang_probs, {}

# Function to save dataframe in chunks to prevent memory issues
def save_df_in_chunks(df, output_file, chunk_size=10000):
    """Save large dataframe in chunks to avoid memory issues."""
    for i in range(0, len(df), chunk_size):
        chunk = df.iloc[i:i + chunk_size]
        mode = 'w' if i == 0 else 'a'
        header = i == 0
        chunk.to_csv(output_file, 
                    mode=mode, 
                    header=header, 
                    index=False, 
                    escapechar='\\', 
                    encoding='utf-8-sig')
        # Clear memory
        del chunk
        gc.collect()
    print(f"Saved {len(df)} samples to {output_file} in chunks")

# Function 1: Load from Public Datasets (OSCAR) with memory optimization
def load_from_datasets(output_file="dataset_corpus.csv"):
    print("Loading from public datasets...")
    all_samples = 0
    oscar_langs = {
        "hi": "Hindi",
        "mr": "Marathi",
        "gu": "Gujarati",
        "bn": "Bengali",
        "ta": "Tamil",
        "kn": "Kannada"
    }
    
    # Clear any existing file
    open(output_file, 'w', encoding='utf-8-sig').close()
    
    for code, lang in oscar_langs.items():
        try:
            # Load and process in smaller chunks
            dataset = load_dataset("oscar", f"unshuffled_deduplicated_{code}", split="train", trust_remote_code=True)
            chunk_size = min(10000, TARGET_PER_LANG)
            num_chunks = min(TARGET_PER_LANG // chunk_size, len(dataset) // chunk_size)
            
            total_lang_samples = 0
            for i in range(num_chunks):
                start_idx = i * chunk_size
                end_idx = min((i + 1) * chunk_size, len(dataset))
                
                chunk_data = dataset[start_idx:end_idx]
                chunk_df = pd.DataFrame(chunk_data)
                filtered_df = chunk_df[chunk_df['text'].apply(filter_text_length)]
                
                if not filtered_df.empty:
                    filtered_df = filtered_df[['text']].assign(language=lang)
                    # Write chunk to file
                    mode = 'a' if i > 0 or all_samples > 0 else 'w'
                    header = i == 0 and all_samples == 0
                    filtered_df.to_csv(output_file, 
                                      mode=mode, 
                                      header=header, 
                                      index=False, 
                                      escapechar='\\', 
                                      encoding='utf-8-sig')
                    
                    samples_added = len(filtered_df)
                    total_lang_samples += samples_added
                    all_samples += samples_added
                
                # Clear memory
                del chunk_data, chunk_df, filtered_df
                gc.collect()
                
                # Stop if we've collected enough samples
                if total_lang_samples >= TARGET_PER_LANG:
                    break
                    
            print(f"Loaded {total_lang_samples} {lang} samples from OSCAR")
            
        except Exception as e:
            print(f"Error loading OSCAR for {lang}: {e}")
    
    print(f"Saved total of {all_samples} samples from datasets to {output_file}")
    return all_samples

# Function 2: Web Scraping with memory optimization
def scrape_from_web(output_file="scraped_corpus.csv"):
    print("Scraping from web...")
    sites = {
        "Hindi": ["https://hindi.bbc.com", "https://www.bhaskar.com"],
        "Marathi": ["https://lokmat.com", "https://maharashtratimes.com"],
        "Sindhi": ["https://awamiawaz.pk"],
        "Gujarati": ["https://divyabhaskar.co.in", "https://sandesh.com"],
        "Bengali": ["https://anandabazar.com", "https://eisamay.com"],
        "Tamil": ["https://dinamalar.com", "https://dailythanthi.com"],
        "Kannada": ["https://prajavani.net", "https://vijaykarnataka.com"]
    }
    
    # Clear any existing file
    open(output_file, 'w', encoding='utf-8-sig').close()
    all_samples = 0
    
    headers = {"User-Agent": "Mozilla/5.0"}
    
    for lang, urls in sites.items():
        lang_texts = []
        max_samples_per_url = TARGET_PER_LANG // len(urls)
        
        for url in urls:
            try:
                response = requests.get(url, headers=headers, timeout=10)
                soup = BeautifulSoup(response.content, "html.parser")
                paragraphs = soup.find_all("p")
                
                for p in paragraphs:
                    if len(lang_texts) >= max_samples_per_url:
                        break
                        
                    text = clean(p.get_text(), no_line_breaks=True, no_urls=True, no_emails=True)
                    if filter_text_length(text):
                        lang_texts.append({"text": text, "language": lang})
                        
                # Save immediately if we have enough data
                if len(lang_texts) >= max_samples_per_url:
                    break
                    
            except Exception as e:
                print(f"Error scraping {url}: {e}")
        
        # Save language data
        if lang_texts:
            lang_df = pd.DataFrame(lang_texts)
            mode = 'a' if all_samples > 0 else 'w'
            header = all_samples == 0
            lang_df.to_csv(output_file, 
                          mode=mode, 
                          header=header, 
                          index=False, 
                          escapechar='\\', 
                          encoding='utf-8-sig')
            
            all_samples += len(lang_df)
            print(f"Scraped {lang}: {len(lang_df)} samples")
            
            # Clear memory
            del lang_df, lang_texts
            gc.collect()
    
    print(f"Saved total of {all_samples} samples from web to {output_file}")
    return all_samples

# Function 3: OCR from PDFs using Kaggle path with memory optimization
def extract_from_pdfs(pdf_dir="/kaggle/input/pdddffs/allpdfs", output_file="pdf_corpus.csv"):
    print("Extracting from PDFs...")
    
    if not os.path.exists(pdf_dir):
        print(f"PDF directory {pdf_dir} not found, skipping...")
        return 0
    
    # Clear any existing file
    open(output_file, 'w', encoding='utf-8-sig').close()
    all_samples = 0
    
    # Limit to 10 PDFs max to prevent memory issues
    pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")][:10]
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_dir, pdf_file)
        pdf_corpus = []
        
        try:
            # Try pdfminer for searchable PDFs first
            text = extract_text(pdf_path)
            if not text.strip():  # If empty, use OCR
                text = extract_text_from_pdf(pdf_path, batch_size=5)  # Reduced batch size
            
            if not text:
                continue
            
            # Split into paragraphs
            paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
            
            for para in paragraphs:
                if filter_text_length(para):
                    top_lang, lang_probs, _ = detect_language_unicode(para)
                    if top_lang in LANGUAGES:  # Only include target languages
                        pdf_corpus.append({"text": para, "language": top_lang})
            
            # Save immediately after each PDF to prevent memory buildup
            if pdf_corpus:
                pdf_df = pd.DataFrame(pdf_corpus)
                mode = 'a' if all_samples > 0 else 'w'
                header = all_samples == 0
                pdf_df.to_csv(output_file, 
                             mode=mode, 
                             header=header, 
                             index=False, 
                             escapechar='\\', 
                             encoding='utf-8-sig')
                
                all_samples += len(pdf_df)
                print(f"Extracted {len(pdf_df)} samples from {pdf_file}")
                
                # Clear memory
                del pdf_df, pdf_corpus
                gc.collect()
            
        except Exception as e:
            print(f"Error processing {pdf_path}: {e}")
        
        # Clear more memory
        text = None
        paragraphs = None
        gc.collect()
    
    print(f"Saved total of {all_samples} samples from PDFs to {output_file}")
    return all_samples

# Main function to build the corpus with memory optimizations
def build_corpus(output_file="indic_corpus.csv"):
    print("Building the corpus...")
    
    # Process each source individually and merge at the end
    dataset_count = load_from_datasets("temp_dataset_corpus.csv")
    gc.collect()  # Force garbage collection
    
    scraped_count = scrape_from_web("temp_scraped_corpus.csv")
    gc.collect()  # Force garbage collection
    
    pdf_count = extract_from_pdfs(output_file="temp_pdf_corpus.csv")
    gc.collect()  # Force garbage collection
    
    # If we have data from any source, merge and balance
    if dataset_count + scraped_count + pdf_count > 0:
        print("Merging and balancing corpus...")
        
        # Create a new output file
        open(output_file, 'w', encoding='utf-8-sig').close()
        
        # Process each language separately
        for lang in LANGUAGES:
            lang_samples = []
            
            # Process each source file in chunks to avoid memory issues
            for source_file in ["temp_dataset_corpus.csv", "temp_scraped_corpus.csv", "temp_pdf_corpus.csv"]:
                if os.path.exists(source_file) and os.path.getsize(source_file) > 0:
                    # Read in chunks
                    chunk_size = 10000
                    for chunk in pd.read_csv(source_file, 
                                            chunksize=chunk_size, 
                                            encoding='utf-8-sig', 
                                            escapechar='\\'):
                        # Filter for current language
                        lang_chunk = chunk[chunk['language'] == lang]
                        if not lang_chunk.empty:
                            lang_samples.append(lang_chunk)
                            
                            # If we have enough samples, stop reading
                            total_samples = sum(len(df) for df in lang_samples)
                            if total_samples >= TARGET_PER_LANG:
                                break
                    
                    # Clear memory
                    gc.collect()
            
            # Combine all chunks for this language
            if lang_samples:
                combined_lang = pd.concat(lang_samples)
                available_samples = len(combined_lang)
                
                # Sample if we have more than needed
                if available_samples > TARGET_PER_LANG:
                    combined_lang = combined_lang.sample(n=TARGET_PER_LANG, random_state=42)
                
                # Save this language to the final file
                mode = 'a' if os.path.getsize(output_file) > 0 else 'w'
                header = mode == 'w'
                combined_lang.to_csv(output_file, 
                                    mode=mode, 
                                    header=header, 
                                    index=False, 
                                    escapechar='\\', 
                                    encoding='utf-8-sig')
                
                print(f"Added {len(combined_lang)} {lang} samples to final corpus")
                
                # Clear memory
                del combined_lang, lang_samples
                gc.collect()
            else:
                print(f"No samples found for {lang}")
        
        # Get final count
        total_lines = 0
        with open(output_file, 'r', encoding='utf-8-sig') as f:
            for _ in f:
                total_lines += 1
        
        total_lines -= 1  # Subtract header line
        print(f"Final corpus saved to {output_file} with {total_lines} samples")
        
        # Clean up temporary files
        for temp_file in ["temp_dataset_corpus.csv", "temp_scraped_corpus.csv", "temp_pdf_corpus.csv"]:
            if os.path.exists(temp_file):
                os.remove(temp_file)
    else:
        print("No data collected!")

if __name__ == "__main__":
    build_corpus()

Building the corpus...
Loading from public datasets...


README.md:   0%|          | 0.00/303k [00:00<?, ?B/s]

oscar.py:   0%|          | 0.00/14.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1909387 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

...

Loaded 214656 Bengali samples from OSCAR


Generating train split:   0%|          | 0/833101 [00:00<?, ? examples/s]

Loaded 190633 Tamil samples from OSCAR


Generating train split:   0%|          | 0/251064 [00:00<?, ? examples/s]

Loaded 177725 Kannada samples from OSCAR
Saved total of 1084169 samples from datasets to temp_dataset_corpus.csv
Scraping from web...
Error scraping https://hindi.bbc.com: HTTPSConnectionPool(host='hindi.bbc.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fcdb933cd90>: Failed to resolve 'hindi.bbc.com' ([Errno -2] Name or service not known)"))
Saved total of 0 samples from web to temp_scraped_corpus.csv
Extracting from PDFs...
Error extracting text from /kaggle/input/pdddffs/allpdfs/170502_46th_BoG_Minutes.pdf: Unable to get page count. Is poppler installed and in PATH?
Error extracting text from /kaggle/input/pdddffs/allpdfs/471 (TO).pdf: Unable to get page count. Is poppler installed and in PATH?
Error extracting text from /kaggle/input/pdddffs/allpdfs/Extension-of-Ahdoc-Employees.pdf: Unable to get page count. Is poppler installed and in PATH?
Error extracting text from /kaggle/input/pdddffs/allpdfs/1

In [6]:
df = pd.read_csv("indic_corpus.csv", encoding='utf-8-sig', escapechar='\\')

In [7]:
df

,text,language
0,"'आइटम गर्ल' बनकर हिट हुई थीं राखी सावंत, आज कर...",Hindi
1,नई दिल्ली: देश में अगस्त में थोक मूल्य सूचकांक...,Hindi
2,वरिष्ठ समाजवादी नेता शरद यादव ने अगले साल लोकस...,Hindi
3,बेंगलुरु के आसमान में इंडिगो के 2 विमान टकराने...,Hindi
4,थोक कारखाने - उच्च वापस निर्माताओं और आपूर्तिक...,Hindi
...,...,...
1084164,ಹಿಂದೂ ಧರ್ಮದಲ್ಲಿ ಸಂಪತ್ತಿನ ದೇವತೆಯೆಂದೇ ಪೂಜಿಸಲ್ಪಡು...,Kannada
1084165,"ಬೆಳಗಾವಿ, ಜೂ.24- ಅನಾರೋಗ್ಯದಿಂದ ಬಳಲಿ ದಯಾಮರಣಕ್ಕೆ ಅ...",Kannada
1084166,ಕೇರಳ ಶಿಕ್ಷಕರ ಅರ್ಹತಾ ಪರೀಕ್ಷೆ (ಕೆ.ಟಿ.ಟಿ.ಟಿ) ಯನ್ನ...,Kannada
1084167,ದೇವದುರ್ಗ: ತಾಲ್ಲೂಕಿನ ಕೆ. ಇರಬಗೇರಾ ಗ್ರಾಮ ಪಂಚಾಯಿತಿ...,Kannada


In [12]:
# Create a dataset
!mkdir -p /kaggle/working/indic_corpus_dataset
!mv /kaggle/working/indic_corpus.csv /kaggle/working/indic_corpus_dataset/
# For compressed files:
# !mv /kaggle/working/indic_corpus.csv.gz /kaggle/working/indic_corpus_dataset/

mv: cannot stat '/kaggle/working/indic_corpus.csv': No such file or directory


In [5]:
!pip install pdfminer pdf2image pytesseract langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 37.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140078 sha256=bda0fdb6600a646b555e5ed93d1c9e7129120340f7c9448f7f63d3a61bffc857
  Stored in directory: /root/.cache/pip/wheels/4e/c1/68/f7bd0a8f514661f76b5cbe3b5f76e0033d79f1296012cbbf72
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=4c195e25d477eb4f75cd664bb338af019bf9b65983870ca1ad9be7c9ca288d39
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built pdfminer langdetect


In [7]:
!pip install pdfminer.six


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.1 MB/s eta 0:00:0000:0100:01


In [9]:
!pip install cleantext

In [13]:
df

,text,language
0,"'आइटम गर्ल' बनकर हिट हुई थीं राखी सावंत, आज कर...",Hindi
1,नई दिल्ली: देश में अगस्त में थोक मूल्य सूचकांक...,Hindi
2,वरिष्ठ समाजवादी नेता शरद यादव ने अगले साल लोकस...,Hindi
3,बेंगलुरु के आसमान में इंडिगो के 2 विमान टकराने...,Hindi
4,थोक कारखाने - उच्च वापस निर्माताओं और आपूर्तिक...,Hindi
...,...,...
1084164,ಹಿಂದೂ ಧರ್ಮದಲ್ಲಿ ಸಂಪತ್ತಿನ ದೇವತೆಯೆಂದೇ ಪೂಜಿಸಲ್ಪಡು...,Kannada
1084165,"ಬೆಳಗಾವಿ, ಜೂ.24- ಅನಾರೋಗ್ಯದಿಂದ ಬಳಲಿ ದಯಾಮರಣಕ್ಕೆ ಅ...",Kannada
1084166,ಕೇರಳ ಶಿಕ್ಷಕರ ಅರ್ಹತಾ ಪರೀಕ್ಷೆ (ಕೆ.ಟಿ.ಟಿ.ಟಿ) ಯನ್ನ...,Kannada
1084167,ದೇವದುರ್ಗ: ತಾಲ್ಲೂಕಿನ ಕೆ. ಇರಬಗೇರಾ ಗ್ರಾಮ ಪಂಚಾಯಿತಿ...,Kannada


0          'आइटम गर्ल' बनकर हिट हुई थीं राखी सावंत, आज कर...
1          नई दिल्ली: देश में अगस्त में थोक मूल्य सूचकांक...
2          वरिष्ठ समाजवादी नेता शरद यादव ने अगले साल लोकस...
3          बेंगलुरु के आसमान में इंडिगो के 2 विमान टकराने...
4          थोक कारखाने - उच्च वापस निर्माताओं और आपूर्तिक...
                                 ...                        
1084164    ಹಿಂದೂ ಧರ್ಮದಲ್ಲಿ ಸಂಪತ್ತಿನ ದೇವತೆಯೆಂದೇ ಪೂಜಿಸಲ್ಪಡು...
1084165    ಬೆಳಗಾವಿ, ಜೂ.24- ಅನಾರೋಗ್ಯದಿಂದ ಬಳಲಿ ದಯಾಮರಣಕ್ಕೆ ಅ...
1084166    ಕೇರಳ ಶಿಕ್ಷಕರ ಅರ್ಹತಾ ಪರೀಕ್ಷೆ (ಕೆ.ಟಿ.ಟಿ.ಟಿ) ಯನ್ನ...
1084167    ದೇವದುರ್ಗ: ತಾಲ್ಲೂಕಿನ ಕೆ. ಇರಬಗೇರಾ ಗ್ರಾಮ ಪಂಚಾಯಿತಿ...
1084168    ಸಂತ ತೆರೇಸಮ್ಮನವರ ದೇವಾಲಯ: ಭಾನುವಾರ ‘ಜೋಸೆಫ್ ಮತ್ತು ...
Name: text, Length: 1084169, dtype: object

In [16]:
!pip install modin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00 0:00:01


In [ ]:
import modin.pandas as pd
df = pd.read_csv("/kaggle/working/dataset_corpus.csv")


2025-03-10 15:14:48,861	INFO worker.py:1841 -- Started a local Ray instance.


In [2]:
import pandas as pd

In [3]:
dd=pd.read_csv("/kaggle/working/indic_corpus.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe0 in position 0: unexpected end of data

In [ ]:
dd